In [1]:
from langchain_community.chat_models import ChatOllama
from openai import OpenAI
import keyring

llm_llama = ChatOllama(model='llama3.1')
llm_openai = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_windows')
)

def get_response_from_llama(prompt):
    response = llm_llama.invoke(prompt)
    return response.content

def get_response_from_openai(prompt):
    completion = llm_openai.chat.completions.create(
        messages=[
            {'role':'user', 'content':prompt}
        ],
        model='gpt-3.5-turbo',
        max_tokens=1024,
        temperature=0.0
    )
    return completion.choices[0].message.content

In [3]:
# 엔티티 링킹
text = "패캠전자는 올해 1분기 매출이 20% 증가했다고 발표했습니다. 반면, 패캠자동차는 올해 1분기 실적이 15% 감소할 것으로 전망했습니다."

prompt = f'''아래 단계 별로 엔티티링킹을 수행하세요.
1단계. 아래 텍스트에서 엔터티를 추출하세요.
2단계. 추출된 엔터티와 데이터베이스의 항목을 연결하세요.

# 데이터베이스
- 패캠전자 : 전자제품 제조업
- 패캠자동차 : 전기자동차 전문 기업

# 텍스트
{text}
'''

answer = get_response_from_llama(prompt)
print(answer)

1단계. 엔터티를 추출하세요.

* 패캠전자 
* 패캠자동차
* 전자제품 제조업
* 전기자동차
* 매출
* 실적

2단계. 엔터티와 데이터베이스의 항목을 연결하세요.

* 패캠전자 : 전자제품 제조업 (연관)
* 패캠전자 : 매출 (연관) -> 실제로는 패캠전자에 관한 텍스트에서 매출이 증가했다는 것을 알 수 있으므로 패캠전자와 매출은 연관된 엔터티입니다.
* 패캠자동차 : 전기자동차 (연관)
* 패캠자동차 : 실적 (연관) -> 실제로는 패캠자동차에 관한 텍스트에서 실적이 감소할 것으로 전망되었다는 것을 알 수 있으므로 패캠자동차와 실적은 연관된 엔터티입니다.


In [5]:
# 엔티티 링킹 및 json 포맷화
text = "패캠전자는 올해 1분기 매출이 20% 증가했다고 발표했습니다. 반면, 패캠자동차는 올해 1분기 실적이 15% 감소할 것으로 전망했습니다."

prompt = f'''아래 단계 별로 엔티티링킹을 수행하세요.
1단계. 아래 텍스트에서 엔터티를 추출하세요.
2단계. 추출된 엔터티와 데이터베이스의 항목을 연결하세요.
출력 포맷은 다음과 같습니다.
{{"<엔티티명>":<엔티티에 대한 데이터베이스 설명>,...}}

# 데이터베이스
- 패캠전자 : 전자제품 제조업
- 패캠자동차 : 전기자동차 전문 기업

# 텍스트
{text}
'''

answer = get_response_from_llama(prompt)
print(answer)

1단계에서 엔터티를 추출하면 다음과 같습니다.

- 패캠전자
- 패캠자동차

2단계에서 추출된 엔터티와 데이터베이스의 항목을 연결하면 다음과 같습니다.

{"패캠전자": "전자제품 제조업", "패캠자동차": "전기자동차 전문 기업"}

이러한 결과가 출력되도록 엔티티링킹이 수행되었습니다.


In [7]:
# 엔티티 링킹 및 감성 분석
text = "패캠전자는 올해 1분기 매출이 20% 증가했다고 발표했습니다. 반면, 패캠자동차는 올해 1분기 실적이 15% 감소할 것으로 전망했습니다."

prompt = f'''아래 단계 별로 엔티티링킹을 수행하세요.
1단계. 아래 텍스트에서 엔터티를 추출하세요.
2단계. 추출된 엔터티와 데이터베이스의 항목을 연결하세요.
3단계. 연결된 엔티티의 감성을 분석하세요.
출력 포맷은 다음과 같습니다.
{{"<엔티티명>":"<긍정/부정/중립>",...}}

# 데이터베이스
- 패캠전자 : 전자제품 제조업
- 패캠자동차 : 전기자동차 전문 기업

# 텍스트
{text}
'''

answer = get_response_from_llama(prompt)
print(answer)

**엔티티링킹 수행 결과**

{"패캠전자":"긍정", "패캠자동차":"부정"}

### 1단계. 엔터티 추출
* 패캠전자 : 전자제품 제조업
* 패캠자동차 : 전기자동차 전문 기업

**추출된 엔터티**

* 패캠전자
* 패캠자동차

### 2단계. 엔터티와 데이터베이스의 항목 연결
* 패캠전자 -> 전자제품 제조업 (패캠전자)
* 패캠자동차 -> 전기자동차 전문 기업 (패캠자동차)

**연결된 엔터티**

* 패캠전자 : 패캠전자
* 패캠자동차 : 패캠자동차

### 3단계. 연결된 엔티티의 감성 분석
패캠전자는 올해 1분기 매출이 20% 증가했다고 발표했습니다.
-> 긍정 (매출 증가)
반면, 패캠자동車는 올해 1분기 실적이 15% 감소할 것으로 전망했습니다.
-> 부정 (실적 감소)

**엔티티의 감성**

* 패캠전자 : 긍정
* 패캠자동차 : 부정

결과적으로, **{"패캠전자":"긍정", "패캠자동차":"부정"}**가 출력됩니다.


In [8]:
# 엔티티 링킹 및 감성 수치화
text = "패캠전자는 올해 1분기 매출이 20% 증가했다고 발표했습니다. 반면, 패캠자동차는 올해 1분기 실적이 15% 감소할 것으로 전망했습니다."

prompt = f'''아래 단계 별로 엔티티링킹을 수행하세요.
1단계. 아래 텍스트에서 엔터티를 추출하세요.
2단계. 추출된 엔터티와 데이터베이스의 항목을 연결하세요.
3단계. 연결된 엔티티의 감성(positive, negative, neutral을 분석하세요.
출력 포맷은 다음과 같습니다.
{{"<엔티티명>": {{"positive":<probability>, "negative":<probability>, "neutral":<probability>}},...}}

# 데이터베이스
- 패캠전자 : 전자제품 제조업
- 패캠자동차 : 전기자동차 전문 기업

# 텍스트
{text}
'''

answer = get_response_from_llama(prompt)
print(answer)

1단계. 엔터티를 추출하세요.

- 패캠전자
- 패캠자동차

2단계. 엔터티와 데이터베이스의 항목을 연결하세요.

- 패캠전자 : 전자제품 제조업
- 패캠자동차 : 전기자동차 전문 기업

3단계. 감성 분석 결과를 다음과 같이 출력합니다.

{"패캠전자": {"positive":0.5, "negative":0, "neutral":0.5}, 
"패캠자동차": {"positive":0, "negative":0.8, "neutral":0.2}}

설명: 

- 패캠전자는 전자제품 제조업이라는 긍정적인 특성을 가지고 있으므로 positive의 확률은 0.5입니다.
- 패캠전자에 대해 음성적 인 언급이 없는 것으로 보아, negative의 확률은 0입니다.
- 패캠전자에 대한 정보가 부족하여 neutral의 확률도 0.5입니다.

- 패캠자동차는 전기자동차 전문 기업이라는 긍정적인 특성을 가지고 있지 않으므로 positive의 확률은 0입니다.
- 패캠자동차에 대해 음성적 인 언급이 많고 실적이 감소할 것으로 전망되었다는 소식이 나와 negative의 확률은 0.8입니다.
- 패캠자동차에 대한 정보가 부족하여 neutral의 확률도 0.2입니다.

위의 결과는 엔터티링킹을 수행한 결과로, 각 엔터티에 대한 감성 분석 결과를 포함하고 있습니다.


In [9]:
# api화
text = "패캠전자는 올해 1분기 매출이 20% 증가했다고 발표했습니다. 반면, 패캠자동차는 올해 1분기 실적이 15% 감소할 것으로 전망했습니다."

prompt = f'''아래 단계 별로 엔티티링킹을 수행하세요.
1단계. 아래 텍스트에서 엔터티를 추출하세요.
2단계. 추출된 엔터티와 데이터베이스의 항목을 연결하세요.
3단계. 연결된 엔티티의 감성(positive, negative, neutral을 분석하세요.
출력 포맷은 다음과 같습니다.
{{"<엔티티명>": {{"positive":<probability>, "negative":<probability>, "neutral":<probability>}},...}}
출력 포맷 이외의 문장은 생성하지 마시오.

# 데이터베이스
- 패캠전자 : 전자제품 제조업
- 패캠자동차 : 전기자동차 전문 기업

# 텍스트
{text}
'''

answer = get_response_from_llama(prompt)
print(answer)

{"패캠전자": {"positive": 0.8, "negative": 0, "neutral": 0.2}, 
"패캠자동차": {"positive": 0, "negative": 0.85, "neutral": 0.15}}
